In [2]:
from telethon import TelegramClient, events, sync
from telethon import functions, types
from bs4 import BeautifulSoup
import lxml
from urllib.request import Request, urlopen
from urllib.parse import urlparse

import re
import os
import itertools
import random
import datetime
import json

import pandas as pd
import dask.dataframe as dd
import numpy as np
from ast import literal_eval

import community
import community as community_louvain
import networkx as nx
import matplotlib.pyplot as plt
from scipy.stats.stats import pearsonr   
from polyglot.detect import Detector
from polyglot.detect.base import UnknownLanguage


from telegraph.scraper.utils import extract_channels_from_links_website
from telegraph.graph.utils import get_json, who_shared, make_graph_from_df, get_pagerank, graph_to_neo4j

from sklearn.feature_extraction.text import CountVectorizer
#import lda

/opt/conda/lib/python3.7/site-packages/dask/dataframe/utils.py:13: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm
/opt/conda/lib/python3.7/site-packages/sklearn/feature_extraction/image.py:167: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dtype=np.int):


# Import Clean Data

In [3]:
# Specify import types for data
dtype = {'channel_id' : float, 
 'channel_name' : str, 
 'forward_from' : str, 
 'forward_from_channel_id' : float,
 'forward_from_post_id' : float,
 'group_type' : str, 
 'id' : float,
 'is_reply' : bool, 
 'links': str,
 'member_count' : float,
 'mentioned_in_link': str,
 'mentions' : str,
 'participants_count' : float,
 'post_author' : str,
 'raw_text' : str,
 'reply_to_message_id' : float,
 'sender' : object,
 'sender_id' :  float,
 'title' : str,
 'views': float}

In [4]:
# directory for imports 
curr_dir = '/root/repos/telegraph'

In [5]:
# Channels with less than 10 messages
channels_less_than_10_messages = ['cbsnews', 'newsmax', 'foxnews', 'realdonaldtrump', 'msagd', 'gruene', 'nuerburgring', 'digitaler', 'potus', 'hilton', 'breitbartnews', 'yahoo', 'freiemedien', 'protonmail', 'tuckercarlson', 'posteo', 'querdenken_7141', 'maske', 'freiheitmachtwahr', 'friede_freiheit_demokratie_aut', 'danscavino', 'chrisaresoffiziell', 'archemovie', 'mike_pence', 'shomburg', 'presssec', 'querdenken_361', 'qanondeutschland', 'proudboysuncensored', 'klappstuhlfruehstueck', 'whitehouse', 'dieregierungmussweg', 'chatderfreiheit', 'klagemauer', 'corona', 'twitter', 'punkt', 'pandemie1', 'querdenken351_aktiv', 'demo_kraten', 'querdenken_831', 'thejusticedept', 'sozmi', 'querdenken_571pw', 'querdenken334', 'exfakt', 'ntd_tv', 'secpompeo', 'querdenken40_aktiv', 'polizei', 'updayde', 'gunnarkaiser', 'rosen', 'scharfschuetzenbewegung', 'impfen', 'querdenken241_aktiv', 'sidneypowell1', 'benny_johnson', 'elternstehenauf', 'soziales', 'daniel_prinz_offiziell', 'infokanalberlin', 'querdenken_40', 'querdenken831', 'querdenken_7451', 'corona_jugend_informiert', 'impfenst', 'querdenken_702', 'd_day2_0_button_verteiler_kanal', 'fakealarm', 'diebasis', 'llinwood', 'fluthilfe_portal', 'flutmanager', 'querdenken571pw_aktiv', 'steven_crowder']

In [6]:
# Get iterations file (Json keeping track when data was scraped)
iterations_path = f'{curr_dir}/data_final/iterations/16-Aug-2021 (08-03-48.974626).json'
    
with open(iterations_path) as f:
    iterations = json.load(f)

# Extract all channels from the first 10 iterations
subset = {k: v for k, v in iterations.items() if k in [str(i) for i in range(11)]}

# Transform to list of channels
channels_total = [v.lower() for k, v in subset.items() for v in list(v.keys())]
    
len(channels_total)

2055

In [7]:
# Clean to small channels 
channels = [channel for channel in channels_total if channel not in channels_less_than_10_messages]
len(channels)

1979

In [11]:
# Create dask df for splitting up dataset
df_paths = ["%s/data_final/csvs/%s.csv" % (curr_dir, channel_name) for channel_name in channels]

df = dd.read_csv(df_paths, parse_dates=['datetime'], dtype=dtype, engine="python", error_bad_lines=False, blocksize=None)

Skipping line 580: unexpected end of data


In [12]:
df

,channel_name,channel_id,title,datetime,participants_count,raw_text,views,id,sender,post_author,is_reply,links,reply_to_message_id,group_type,member_count,forward_from,forward_from_channel_id,forward_from_post_id,mentions,sender_id,mentioned_in_link
npartitions=1979,,,,,,,,,,,,,,,,,,,,,
,object,float64,object,"datetime64[ns, UTC]",float64,object,float64,float64,object,object,bool,object,float64,object,float64,object,float64,float64,object,float64,object
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


# Helper functions and dataframes for fast data processing

In [10]:
%%time
# Create fast in memory subset dataframe for fast analysis
df_pd = df[["channel_id", "datetime", "participants_count", "links", "group_type", "forward_from", "mentions", "views"]].compute()
d = {'group': True, 'channel': False}
df_pd= df_pd.assign(is_group = lambda x: x.group_type.map(d)).drop(columns=['group_type'])

pickle_path = f'{curr_dir}/data_final/'

df_pd.to_pickle(pickle_path + "data.pkl")

# Create df to merge id back to name
pairs = df[["channel_name", "channel_id"]].drop_duplicates(subset=["channel_name"]).compute()
pairs.to_pickle(pickle_path + "pairs.pkl")

CPU times: user 29min 53s, sys: 1min 18s, total: 31min 12s
Wall time: 30min 10s


In [8]:
# Load df_pd lazy sometimes
pickle_path = f'{curr_dir}/data_final/'
df_pd = pd.read_pickle(f'{pickle_path}/data.pkl')

In [9]:
%%time
# Add links list
links = df_pd[["channel_id", "datetime", "forward_from", "links"]].assign(links=df_pd.links.apply(literal_eval)).explode("links").dropna(subset = ['links'])

# Get domain
def get_domain(x):
    try: 
        return urlparse(str(x)).hostname
    except ValueError:
        return None

links_domain = links.assign(domain = lambda x: [get_domain(x) for x in x.links])

links_domain.to_pickle(pickle_path + "links.pkl")
links_domain

CPU times: user 6min 2s, sys: 1min 10s, total: 7min 12s
Wall time: 7min 13s


In [53]:
# Helper Functions
def get_id(name):
    return pairs[pairs.channel_name == name].channel_id.iloc[0]

def get_name(channel_id):
    return pairs[pairs.channel_id == channel_id].channel_name.iloc[0]

# Add dataframe metadata (Community, Pagerank, size etc.)

In [17]:
# Calcuclate edges and weights
edges_df = (df_pd
            .loc[:, ["channel_id", "forward_from", "participants_count"]]
            .groupby(by=["channel_id", "forward_from"])
            .count()
            .reset_index()
            .merge(pairs, left_on="channel_id", right_on="channel_id")
            .drop(columns=["channel_id"])
            .reindex(columns=["channel_name", "forward_from", "participants_count"]))

edges_tuple = [tuple(x) for x in edges_df.to_numpy()]

# Only core network (channels that have been scraped)
edges_tuple_only_scraped = [ele for ele in edges_tuple if ele[0] in channels and ele[1] in channels]

In [18]:
# Create networkx graph
G = nx.DiGraph()
G.add_weighted_edges_from(edges_tuple_only_scraped)

In [21]:
# Calculate pagerank
pagerank_dict = nx.pagerank(G)
pagerank = pd.DataFrame.from_dict(pagerank_dict, orient='index', columns=["rank"]).sort_values(by="rank", ascending=False)

In [76]:
# Undirected Graph for community detection
G_und = G.to_undirected()

# Compute partition
partition = community.best_partition(G_und, random_state=1)

# Save communities
community_df = pd.DataFrame(partition.items(), columns=["channel_name", "community"])

In [78]:
# Find languages of channels
channel_languages = []

for index, channel_name in enumerate(channels):
    print(f'Channel {channel_name} out of {index + 1}/{len(channels)} channels is being scraped', end="\r")
    channel_msgs = ' '.join(pd.read_csv('%s/data_final/csvs/%s.csv' % (curr_dir, channel_name)).raw_text.sample(n=200, random_state=1, replace=True).astype(str))
    printable_str = ''.join(x for x in channel_msgs if x.isprintable())
    text = re.sub(r'(?:(?:http|https):\/\/)?([-a-zA-Z0-9.]{2,256}\.[a-z]{2,4})\b(?:\/[-a-zA-Z0-9@:%_\+.~#?&//=]*)?', '', printable_str, flags=re.MULTILINE)
    
    try: 
        detector = Detector(text)
        language = detector.language.code
    except UnknownLanguage:
        print("Language cannot be detected")
        language = np.nan
    
    channel_languages.append([channel_name, language])

Language cannot be detectedout of 94/1979 channels is being scrapedddedcraped


/opt/conda/lib/python3.7/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (9) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Detector is not able to detect the language reliably.


Detector is not able to detect the language reliably.


Detector is not able to detect the language reliably.


...

Detector is not able to detect the language reliably.


Language cannot be detected/1979 channels is being scrapedis being scrapedscraped
Language cannot be detected979 channels is being scraped channels is being scraped


Detector is not able to detect the language reliably.


Language cannot be detectedof 1760/1979 channels is being scrapedapedcraped


Detector is not able to detect the language reliably.


In [39]:
# Get channels from querdenken verzeichnis to classify automatically
req = Request("https://app.querdenken-711.de/initiatives-directory")
html_page = urlopen(req)

soup = BeautifulSoup(html_page, "html.parser")

links = []
for link in soup.findAll('a'):
    links.append(link.get('href'))
    
querdenken_channels_scraped =  extract_channels_from_links_website(links)
querdenken_channels_exisiting = [channel for channel in querdenken_channels_scraped if channel in channels]
querdenken_channels_df = (pd.DataFrame(querdenken_channels_exisiting, columns=["channel_name"])
                          .assign(category_content="querdenker"))
querdenken_channels_df

,channel_name,category_content
0,querdenken_201,querdenker
1,querdenken201,querdenker
2,querdenken201_aktiv,querdenker
3,querdenken203,querdenker
4,querdenken_211,querdenker
...,...,...
148,querdenken841,querdenker
149,querdenken841_aktiv,querdenker
150,querdenken_911,querdenker
151,querdenken911,querdenker


In [84]:
# Bring everything togehter for a channels_metadata list

community_df = pd.DataFrame(partition.items(), columns=["channel_name", "community"])
metadata_df = df_pd.drop_duplicates("channel_id")[["channel_id", "participants_count", "is_group"]]
languages_df = pd.DataFrame(channel_languages, columns=["channel_name", "language"])

channels_df = (pairs
            .assign(link=lambda x: [f'https://t.me/s/{channel_name}' for channel_name in x.channel_name])
            .merge(pagerank.reset_index(), left_on="channel_name", right_on="index", how="outer")
            .merge(community_df, on="channel_name", how="outer")
            .merge(metadata_df, on="channel_id", how="outer")
            .merge(languages_df, on="channel_name", how="outer")
            .merge(querdenken_channels_df, on="channel_name", how="outer")
            .assign(category_type=np.nan))

# Add and clean channel categories
channels_meta = pd.read_csv(f'{pickle_path}communities_meta.csv')

channels_meta.loc[channels_meta.category_content == 'spititual ', "category_content"] = 'spiritual' 
channels_meta.loc[channels_meta.category_content == 'spiritual ', "category_content"] = 'spiritual'
channels_meta.loc[channels_meta.category_content == 'covid-sceptic', "category_content"] = 'spiritual'
channels_meta.loc[channels_meta.category_content == 'conspiracy ', "category_content"] = 'conspiracy'

channels_meta.loc[channels_meta.category_type == 'news-aggregator', "category_type"] = 'news_aggregator'
channels_meta.loc[channels_meta.category_type == 'party', "category_type"] = "political_actor"
channels_meta.loc[channels_meta.category_type == 'organizer'] = "political_initative"

channels_df = (channels_df
             .drop(columns=["category_content", "category_type"])
             .merge(channels_meta[["channel_name", "category_content", "category_type"]], on="channel_name", how="left"))


channels_df.to_csv(f'{curr_dir}/data_final/metadata.csv')
channels_df.to_pickle(f'{pickle_path}metadata.pkl')

channels_df

,channel_name,channel_id,link,index,rank,community,participants_count,is_group,language,category_content,category_type
0,fakepandemie,1.317450e+09,https://t.me/s/fakepandemie,fakepandemie,0.000089,3.0,333.0,False,de,NaN,NaN
1,querdenken7451,1.188593e+09,https://t.me/s/querdenken7451,querdenken7451,0.000089,3.0,14.0,True,de,NaN,NaN
2,querdenken791,1.298666e+09,https://t.me/s/querdenken791,querdenken791,0.000089,3.0,379.0,True,de,querdenker,NaN
3,querdenken_30,1.366609e+09,https://t.me/s/querdenken_30,querdenken_30,0.000105,3.0,733.0,False,de,querdenker,NaN
4,querdenken351,1.417909e+09,https://t.me/s/querdenken351,querdenken351,0.000089,3.0,515.0,True,de,querdenker,NaN
...,...,...,...,...,...,...,...,...,...,...,...
1974,rwelt,1.086141e+09,https://t.me/s/rwelt,rwelt,0.000097,3.0,226.0,False,de,NaN,NaN
1975,nwointelligence,1.358481e+09,https://t.me/s/nwointelligence,nwointelligence,0.000747,0.0,5404.0,False,en,NaN,NaN
1976,emmanuelmacron,1.075445e+09,https://t.me/s/emmanuelmacron,emmanuelmacron,0.000089,0.0,14089.0,False,fr,NaN,NaN
1977,antoinerichard_official,1.279255e+09,https://t.me/s/antoinerichard_official,antoinerichard_official,0.000138,3.0,9642.0,False,de,NaN,NaN


In [79]:
pickle_path = f'{curr_dir}/data_final/'
channels_meta = pd.read_csv(f'{pickle_path}communities_meta.csv')
channels_df = pd.read_pickle(f'{pickle_path}/metadata.pkl')

In [80]:
channels_meta.loc[channels_meta.category_content == 'spititual ', "category_content"] = 'spiritual' 
channels_meta.loc[channels_meta.category_content == 'spiritual ', "category_content"] = 'spiritual'
channels_meta.loc[channels_meta.category_content == 'covid-sceptic', "category_content"] = 'spiritual'
channels_meta.loc[channels_meta.category_content == 'conspiracy ', "category_content"] = 'conspiracy'

channels_meta.loc[channels_meta.category_type == 'news-aggregator', "category_type"] = 'news_aggregator'
channels_meta.loc[channels_meta.category_type == 'party', "category_type"] = "political_actor"
channels_meta.loc[channels_meta.category_type == 'organizer'] = "political_initative"

In [91]:
new_df = (channels_df
 .drop(columns=["category_content", "category_type"])
 .merge(channels_meta[["channel_name", "category_content", "category_type"]], on="channel_name", how="left"))

In [93]:
new_df.to_pickle(f'{pickle_path}metadata.pkl')

In [65]:
channels_meta.category_type.unique()

array(['media', 'influencer', 'news_aggregator', 'party', nan,
       'organizer', 'other', 'telegram-media'], dtype=object)

In [35]:
channels_meta.category_type.value_counts()

news_aggregator    10
influencer          8
media               7
other               4
spiritual           3
telegram-media      2
news-aggregator     2
organizer           1
party               1
conspiracy          1
Name: category_type, dtype: int64

In [ ]:
print([get_name(channel_id) for channel_id in channels_size[channels_size.participants_count < 10].channel_id])

## Deprecated Code

In [ ]:
curr_dir = '/root/repos/telegraph'

In [ ]:
from pandas.errors import EmptyDataError

# Deprecated
# Debug 
csv_paths = ['%s/csv_data/%s' % (curr_dir, pos_csv) for pos_csv in os.listdir(curr_dir + "/csv_data") if pos_csv.endswith('.csv')]

error_files = []
for i, curr_path in enumerate(csv_paths):
    print(f'channel {i}/{len(csv_paths)}', end='\r')
    try: 
        curr = pd.read_csv(curr_path, low_memory=False)
        if len(curr.columns) != 21:
            print(curr_path)
    except EmptyDataError:
        error_files.append(curr_path)
        print(curr_path)

In [ ]:
# Deprecated
# Get all exisiting channels 
p = re.compile('^.*(?=(\.csv))')
existing_channels = [p.match(channel).group() for channel in os.listdir(curr_dir + "/csv_data/") if p.match(channel) is not None]

# Get all existing channels from iteration 
c = [channel for channel in get_n_iterations_channels(11) if channel in existing_channels]
len(c)

# Import all data 
df_all = dd.read_csv(f'{curr_dir}/csv_data/*', parse_dates=['datetime'], dtype=dtype, engine="python", error_bad_lines=False, blocksize=None)

# Get all exisiting channels 
p = re.compile('^.*(?=(\.csv))')
existing_channels = [p.match(channel).group() for channel in os.listdir(curr_dir + "/csv_data/") if p.match(channel) is not None]

# Get all existing channels from iteration 
channels = [channel for channel in get_n_iterations_channels(11) if channel in existing_channels]
len(channels)



In [ ]:
# Get iterations file (Json keeping track when data was scraped)
iterations_path = f'{curr_dir}/data_final/iterations/16-Aug-2021 (08-03-48.974626).json'
    
with open(iterations_path) as f:
    iterations = json.load(f)

# Extract all channels from the first 10 iterations
subset = {k: v for k, v in iterations.items() if k in [str(i) for i in range(11)]}

# Transform to list of channels
channels = [v.lower() for k, v in subset.items() for v in list(v.keys())]
    
channels[1:5]


In [ ]:
import random

# This is a function which resturns random channels for testing of code 
def get_df(amount_channels):
    sample = random.sample(existing_channels, amount_channels)
    df_paths_sample = ["%s/csv_data/%s.csv" % (curr_dir, channel_name) for channel_name in sample]
    return dd.read_csv(df_paths_sample, parse_dates=['datetime'], dtype=dtype, engine="python", error_bad_lines=False, blocksize=None)
    

In [ ]:
# "+".join(["x[%s]" % c for c in list(pd.DataFrame(channels_df.community.value_counts()).query("community < 23").index)])


community_iteration = (pd.DataFrame([[k, channel]
                                    for k,v in iterations.items() 
                                    for channel, count in v.items()], columns=["iteration", "channel_name"])
                     .merge(channels_df)[["iteration", "community", "channel_name"]]
                     .groupby(["iteration", "community"])
                     .count()
                     .rename(columns={"channel_name":"count"})
                     .reset_index()
                     .pivot(index='iteration', columns='community', values='count')
                     .fillna(0)
                     .assign(other=lambda x: x[10.0]+x[16.0]+x[21.0]+x[15.0]+x[12.0]+x[14.0]+x[18.0]+x[19.0]+x[24.0]+x[5.0]+x[27.0]+x[22.0]+x[20.0]+x[13.0]+x[17.0]+x[9.0]+x[7.0]+x[26.0]+x[23.0]+x[28.0]+x[25.0]+x[4.0])
                     [[3.0,1.0,0.0,2.0,8.0,6.0,11.0, "other"]]
                       .apply(lambda x: x/x.sum(), axis=1))


community_iteration.index = community_iteration.index.astype(int)
community_iteration = community_iteration.sort_index()


In [ ]:
# Create meta df 
edges_df = (df_pd
            .loc[:, ["channel_id", "forward_from", "participants_count"]]
            .groupby(by=["channel_id", "forward_from"])
            .count()
            .reset_index()
            .merge(pairs, left_on="channel_id", right_on="channel_id")
            .drop(columns=["channel_id"])
            .reindex(columns=["channel_name", "forward_from", "participants_count"]))

edges_df



In [ ]:
edges_tuple = [tuple(x) for x in edges_df.to_numpy()]

# Only Core Network (Channels that have been scraped)
edges_tuple_only_scraped = [ele for ele in edges_tuple if ele[0] in channels and ele[1] in channels]
edges_tuple_only_scraped[1:5]

In [ ]:
G = nx.DiGraph()
G.add_weighted_edges_from(edges_tuple_only_scraped)

In [ ]:
# Wichtigste Kanäle nach Pagerank. 
pagerank = get_pagerank(G)
pagerank